In [50]:
import numpy as np
import pandas as pd
import re
import jdatetime
import datetime
import calendar

In [2]:
sale_df = pd.read_excel('P2-SalesData.xlsx')
sale_df.head()

,Date,Sales
0,14010115,1395.898445
1,14010114,1654.535199
2,14010113,767.028903
3,14010112,1323.005591
4,14010111,1294.524908


In [3]:
sale_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1111 entries, 0 to 1110
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1111 non-null   int64  
 1   Sales   1111 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 17.5 KB


In [23]:
def to_gregorian(datetime):
    datetime = int(datetime)
    return jdatetime.date(datetime//10000, datetime//100%100, datetime%100).togregorian()

In [25]:
sale_df['Date'] = sale_df['Date'].apply(to_gregorian)

In [28]:
sale_df['Date'] = pd.to_datetime(sale_df['Date'])
sale_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1111 entries, 0 to 1110
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1111 non-null   datetime64[ns]
 1   Sales   1111 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 17.5 KB


In [51]:
def week_of_month(tgtdate):
    days_this_month = calendar.mdays[tgtdate.month]
    for i in range(1, days_this_month):
        d = datetime.datetime(tgtdate.year, tgtdate.month, i)
        if d.day - d.weekday() > 0:
            startdate = d
            break
    # now we canuse the modulo 7 appraoch
    return (tgtdate - startdate).days //7 + 1

In [54]:
# Create Cols
sale_df['Month Day'] = sale_df['Date'].dt.day
sale_df['Week Day'] = sale_df['Date'].dt.weekday
sale_df['Month'] = sale_df['Date'].dt.month
sale_df['Month Week'] = sale_df['Date'].apply(week_of_month)
sale_df['Year Week'] = sale_df['Date'].dt.weekofyear
sale_df['Year Day'] = sale_df['Date'].dt.dayofyear

C:\Users\my name\AppData\Local\Temp\ipykernel_8080\958834914.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  sale_df['Year Week'] = sale_df['Date'].dt.weekofyear


In [55]:
sale_df

,Date,Sales,Month Day,Week Day,Month,Month Week,Year Week,Year Day
0,2022-04-04,1395.898445,4,0,4,1,14,94
1,2022-04-03,1654.535199,3,6,4,0,13,93
2,2022-04-02,767.028903,2,5,4,0,13,92
3,2022-04-01,1323.005591,1,4,4,0,13,91
4,2022-03-31,1294.524908,31,3,3,4,13,90
...,...,...,...,...,...,...,...,...
1106,2019-03-25,1314.944083,25,0,3,4,13,84
1107,2019-03-24,1278.500591,24,6,3,3,12,83
1108,2019-03-23,1316.051583,23,5,3,3,12,82
1109,2019-03-22,1281.464259,22,4,3,3,12,81


In [57]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.layers import LSTM
from sklearn.model_selection import KFold, cross_val_score, train_test_split

In [81]:
X_train = sale_df.drop(axis=1, labels=['Sales', 'Date'])
y_train = sale_df['Sales']

In [82]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(X_train)

In [83]:
X_train.values

array([[ 4,  0,  4,  1, 14, 94],
       [ 3,  6,  4,  0, 13, 93],
       [ 2,  5,  4,  0, 13, 92],
       ...,
       [23,  5,  3,  3, 12, 82],
       [22,  4,  3,  3, 12, 81],
       [21,  3,  3,  3, 12, 80]], dtype=int64)

In [84]:
X_train = X_train.values.reshape(X_train.values.shape[0], X_train.values.shape[1])
X_train = scaler.transform(X_train)

c:\Users\my name\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [88]:
model = Sequential()
model.add(LSTM(4, batch_input_shape= (1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=1, shuffle=False)

IndexError: tuple index out of range